In [ ]:
%%HTML
<!-- Make fonts readable at 1024x768 -->
<style>
.rendered_html { font-size:0.5em; }
</style>

In [ ]:
import numpy as np
import scipy.signal
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pylab as plt
from matplotlib import animation, patches
from IPython.display import display, Audio, HTML
import soundfile as sf
from style import *

### Universidad Austral de Chile 
## INFO185 Comunicaciones


# Unidad 2 - Análisis de señales bidimensiones 

### Dr. Pablo Huijse, phuijse at inf dot uach dot cl 

### <a href="https://github.com/phuijse/UACH-INFO185"> github.com/phuijse/UACH-INFO185 </a>


## Contenidos
- Imágenes, percepción y modelos de color
- Transformada de Fourier bidimensional
- Filtros y convolución
- Transformada coseno bidimensional
- Compresión con estándar JPEG

## Imágenes

- Una imagen se representa como una matriz de NxM componentes. 
- Los elementos de la matriz se llaman píxeles. 
- Los píxeles pueden ser unidimensionales (imagen en blanco y negro) o multidimensionales (RGB, HSV, HSL)
- Típicamente los canales se codifican como un número entero (sin signo) de 8 bits [0, 255] o un número flotante entre [0.0, 1.0]

<center><img src="images/image_matrix.png" width="600"></center>


## Imágenes como señal bidimensional



In [ ]:
from mpl_toolkits.mplot3d import axes3d
x = np.linspace(-2*np.pi, 2*np.pi, 100)
X, Y = np.meshgrid(x, x); Z = 127.5 + 127.5*np.cos(X);
fig = plt.figure(figsize=(10, 10));
ax = fig.add_subplot(111, projection='3d'); ax.set_proj_type('ortho')
def update(angle1=45, angle2=45):
    ax.cla();
    im = ax.plot_surface(X, Y, Z, cmap=plt.cm.RdBu_r); 
    ax.view_init(angle1, angle2); 
    ax.set_xlim3d([-2*np.pi, 2*np.pi]); ax.set_ylim3d([-2*np.pi, 2*np.pi]);
    ax.set_aspect('equal'); #plt.colorbar(im); 
interact(update, angle1=FloatSlider(min=0.0, max=90.0, value=45, 
                               description="Pitch", layout=slider_layout),
            angle2=FloatSlider(min=0.0, max=90.0, value=45, 
                               description="Rotation", layout=slider_layout));

## Sensor Charged Coupled Device (CCD)

- **Efecto fotoeléctrico:** Fotones al impactar un conductor con una cierta frecuencia provocan el desprendimiento de electrones
- En un sensor CCD, cada pixel "cuenta fotones" usando el efecto fotoeléctrico
- La cantidad de cuentas se relaciona con la intensidad. 
- Un conteo se modela como un proceso de Poisson. Si la cantidad de cuentas es grande se puede aproximar con estadísticas Gaussianas.
- El sensor CCD es la base de las cámaras digitales modernas


<center><img src="images/photons-hit-silicon.jpg" width="600"></center>

Ref: https://en.wikipedia.org/wiki/Charge-coupled_device

## Manipulación de imágenes con python

- Podemos leer imágenes usando la función imread de matplotlib 
- Retornará un arreglo tridimensional (filas, columnas, canales)
- Los canales pueden ser 3 (RGB) o 4 (RGBA)
- Podemos operar sobre el arreglo usando numpy, scipy u otras herramientas

In [ ]:
plt.figure(figsize=(10, 5))
img_color = plt.imread('images/valdivia.jpg')
print("Shape: %s, Type: %s" %(repr(img_color.shape), img_color.dtype))
print(img_color[0, 0, :])
plt.imshow(img_color);

Los tres canales para la imagen anterior:

In [ ]:
from matplotlib.colors import LinearSegmentedColormap


custom_cmap = LinearSegmentedColormap.from_list('RedBlack', [(0, 0, 0), (1, 0, 0)], N=100)
plt.figure(figsize=(10, 5)); plt.title('R')
plt.imshow(img_color[:, :, 0], cmap=custom_cmap); plt.colorbar(orientation='vertical')

custom_cmap = LinearSegmentedColormap.from_list('GreenBlack', [(0, 0, 0), (0, 1, 0)], N=100)

plt.figure(figsize=(10, 5)); plt.title('G')
plt.imshow(img_color[:, :, 1], cmap=custom_cmap); plt.colorbar(orientation='vertical')

custom_cmap = LinearSegmentedColormap.from_list('BlueBlack', [(0, 0, 0), (0, 0, 1)], N=100)

plt.figure(figsize=(10, 5)); plt.title('B')
plt.imshow(img_color[:, :, 2], cmap=custom_cmap); plt.colorbar(orientation='vertical');


## Visión humana

- El ojo humano tiene en su retina dos tipos de fotoreceptores: conos y bastones
- Bastones 
    - ~120 millones en la retina
    - No perciben color
    - Requieren poco brillo para producir una señal
    - Tienen baja agudeza (menos sencibles a los detalles)
- Los conos 
    - ~6 millones en la retina
    - Perciben color 
    - Requieren mucho brillo para producir una señal
    - Tienen alta agudeza visual
- Tres tipos de conos cada uno sintonizado a una longitud de onda distinta
<center><img src="images/cones_rods.gif" width="500"></center>
<center><img src="images/cone_rod_density.gif" width="500"></center>

Ref: http://www.danielgmurphy.com/physics/1_intro/contents_phyics1.html

Podemos convertir una imagen en color a escala de grises usando la siguiente transformación

$$
0.299 R + 0.587 G + 0.114 B
$$

que se obtiene del diagrama anterior

In [ ]:
def color2bw(img):
    return np.dot(img, [0.299, 0.587, 0.114]) 

img_bw = to_bw(img_color)
plt.figure(figsize=(10, 5)); 
plt.imshow(img_bw, cmap=plt.cm.Greys_r);

El estándar HSV (Hue, Saturation, Value)

<center><img src="images/hsv_cilinder.jpg" width="400"></center>

La imagen anterior en este estándar

In [ ]:
from matplotlib import colors

img_color = plt.imread('images/valdivia.jpg')
img_color = colors.rgb_to_hsv(img_color)
print("Shape: %s, Type: %s" %(repr(img_color.shape), img_color.dtype))
print(img_color[0, 0, :])

plt.figure(figsize=(10, 5)); plt.title('H')
plt.imshow(img_color[:, :, 0], plt.cm.hsv); plt.colorbar(orientation='vertical')
plt.figure(figsize=(10, 5)); plt.title('S')
plt.imshow(img_color[:, :, 1], plt.cm.Greys_r); plt.colorbar(orientation='vertical')
plt.figure(figsize=(10, 5)); plt.title('V')
plt.imshow(img_color[:, :, 2], plt.cm.Greys_r); plt.colorbar(orientation='vertical');

## Transformada de Fourier bidimensional

La DFT se puede aplicar a funciones multi-dimensionales. En el caso discreto de una señal bidimensional $g[n_1, n_2]$ con índices $n_1 \in [0, N_1-1]$ y $n_2 \in [0, N_2-1]$ tenemos

$$
G[k_1, k_2] = \sum_{n_1=0}^{N_1-1} \sum_{n_2=0}^{N_2-1} g[n_1, n_2] \exp \left ( -j2\pi  \left[\frac{n_1 k_1}{N_1} + \frac{n_2 k_2}{N_2} \right] \right)
$$
y su inversa

$$
g[n_1, n_2] = \frac{1}{N_1 N_2}\sum_{k_1=0}^{N_1-1} \sum_{k_2=0}^{N_2-1} G[k_1, k_2] \exp \left ( j2\pi  \left[\frac{n_1 k_1}{N_1} + \frac{n_2 k_2}{N_2} \right] \right)
$$

Notemos que

\begin{align}
G[k_1, k_2] &= \sum_{n_1=0}^{N_1-1} \left(\sum_{n_2=0}^{N_2-1} g[n_1, n_2] \exp \left (-j2\pi \frac{n_2 k_2}{N_2}\right) \right) \exp \left (-j2\pi \frac{n_1 k_1}{N_1}\right) \\
&= \sum_{n_1=0}^{N_1-1} \gamma_{n_1}[n_2] \exp \left (-j2\pi \frac{n_1 k_1}{N_1}\right),
\end{align}

*i.e.* se descompone como dos DFT de una dimensión. En cada paso podemos usar la FFT

## Base de Fourier en dos dimensiones

- Parte real

In [ ]:
x = np.arange(0, 32, step=1)
X, Y = np.meshgrid(x, x)
fig, ax = plt.subplots(9, 9, figsize=(11, 11), tight_layout=False)
for n in range(9):
    for m in range(9):
        ax[n, m].matshow(np.cos(2.0*np.pi*X*m/len(x) + 2.0*np.pi*Y*n/len(x)), 
                         cmap=plt.cm.RdBu_r, vmin=-1, vmax=1)
        ax[n, m].axis('off')

- Parte imaginaria

In [ ]:
fig, ax = plt.subplots(9, 9, figsize=(11, 11), tight_layout=False)
for n in range(9):
    for m in range(9):
        ax[n, m].matshow(np.sin(2.0*np.pi*X*m/len(x) + 2.0*np.pi*Y*n/len(x)), 
                         cmap=plt.cm.RdBu_r, vmin=-1, vmax=1)
        ax[n, m].axis('off')

## Espectro de una imagen 

- Podemos usar la transformada de Fourier 2D para obtener el espectro de amplitud de una imagen
- Las dimensiones de la DFT son frecuencias espaciales

In [ ]:
from scipy import fftpack
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'

img_bw = color2bw(img_color)
fig, ax = plt.subplots(1, 2, figsize=(11, 4), tight_layout=True)
ax[0].imshow(img_bw); ax[0].set_title("Imagen")
ax[1].set_title("Espectro de amplitud")
ax[1].imshow(fftpack.fftshift(np.abs(fftpack.fft2(img_bw))), 
          extent=[-img_bw.shape[1]//2, img_bw.shape[1]//2, -img_bw.shape[0]//2, img_bw.shape[0]//2]);


- Para visualizar mejor el espectro de una imagen natural se recomienda usar
$$
\log(|\text{fft2}(I)|+1)
$$
de esta forma el componente central no es tan dominante

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11, 4), tight_layout=True)

ax[0].imshow(img_bw); ax[0].set_title("Imagen")
ax[1].set_title("Espectro de amplitud")
ax[1].imshow(fftpack.fftshift(np.log(np.abs(fftpack.fft2(img_bw))+1)), 
          extent=[-img_bw.shape[1]//2, img_bw.shape[1]//2, -img_bw.shape[0]//2, img_bw.shape[0]//2]);
plt.tight_layout(pad=0.1)

## Espectro de una imagen sintética

- Para aprender a interpretar el espectro es útil estudiar imágenes sintéticas
- Por ejemplo: Dos impulsos en el espectro es una sinusoide en el espacio original y viceversa

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True); 

def update(x_pos=0, y_pos=0):
    S_img = np.zeros(shape=(80, 80)); 
    S_img[x_pos, y_pos] = 1.0; S_img[-x_pos, -y_pos] = 1.0;
    #S_img[-x_pos, y_pos] = 1.0; S_img[x_pos, -y_pos] = 1.0; 
    ax[1].set_title("Espectro de amplitud");  ax[0].set_title("Imagen")
    im = ax[1].imshow(fftpack.fftshift(S_img), extent=[-40, 40, 40, -40])
    im = ax[0].imshow(np.real(fftpack.ifft2(S_img))); 
    #plt.colorbar(im, orientation='horizontal')
interact(update, x_pos=IntSlider_nice(min=-39, max=39, value=0, description="x position"),
            y_pos=IntSlider_nice(min=-39, max=39, value=0, description="y position"));

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True); 

def update(x_f=0, y_f=0):
    x = np.arange(0, 80)
    X, Y = np.meshgrid(x, x)
    img = np.cos(2.0*np.pi*X*x_f/80 + 2.0*np.pi*Y*y_f/80)
    # img = np.cos(2.0*np.pi*X*x_f/80) + np.cos(2.0*np.pi*Y*y_f/80)
    # img = np.cos(2.0*np.pi*X*x_f/80)*np.cos(2.0*np.pi*Y*y_f/80)
    S_img = np.abs(fftpack.fft2(img))    
    ax[1].set_title("Espectro de amplitud");  ax[0].set_title("Imagen")
    im = ax[1].imshow(fftpack.fftshift(S_img), extent=[-40, 40, 40, -40])
    im = ax[0].imshow(img); 
    
interact(update, x_f=IntSlider_nice(min=0, max=39, value=0, description="x frequency"),
         y_f=IntSlider_nice(min=0, max=39, value=0, description="y frequency"));

## Espectro de una imagen sintética

- Una línea en la imagen es una línea en el espectro (con otra orientación) ¿Por qué?

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True); 

def update(pix_angle=0):
    img = np.zeros(shape=(80, 80));     
    for i in range(80):
        img[i, int(i - 2*pix_angle*i/80 + pix_angle)] = 1 
    ax[1].set_title("Magnitude spectrum");  ax[0].set_title("Image")
    S_img = np.abs(fftpack.fft2(img))
    im = ax[1].imshow(fftpack.fftshift(S_img), 
                      extent=[-40, 40, 40, -40])
    im = ax[0].imshow(img); 
    
interact(update, pix_angle=IntSlider_nice(min=0, max=79, value=40));

¿Cómo se explica esto? Consideremos el caso donde la rotación es 40 pixeles. 

### Propiedad: 

En ese caso la transformada de Fourier 2D es totalmente separable en dos transformadas de Fourier

In [ ]:
A = np.array([[0, 0, 1, 0, 0]])
B = np.array([[1, 1, 1, 1, 1]])
A*B.T

$$
\begin{align}
G[k_1, k_2] &= \sum_{n_1=0}^{N_1-1} \sum_{n_2=0}^{N_2-1} g[n_1, n_2] \exp \left ( -j2\pi  \left[\frac{n_1 k_1}{N_1} + \frac{n_2 k_2}{N_2} \right] \right) \nonumber \\
& = \sum_{n_1=0}^{N_1-1} \sum_{n_2=0}^{N_2-1} g_1[n_1] g_2[n_2] \exp \left ( -j2\pi \frac{n_1 k_1}{N_1} \right)  \exp \left ( -j2\pi\frac{n_2 k_2}{N_2}  \right) \nonumber \\
& = \sum_{n_1=0}^{N_1-1} g_1[n_1] \exp \left ( -j2\pi \frac{n_1 k_1}{N_1} \right)  \sum_{n_2=0}^{N_2-1} g_2[n_2] \exp \left ( -j2\pi\frac{n_2 k_2}{N_2}  \right) \nonumber \\
\end{align}
$$

- Transformada de Fourier de un impulso: Una constante
- Transformada de Fourier de una constante: Un impulso
Resultado: Una linea rotada en 90º con respecto a la original

Concentremos ahora en los ángulos (en píxeles) distintos de 0, 40 y 80 

¿A qué corresponde el efecto  observado?

### Propiedad: La DFT es periódica

Cuando los ángulos no calzan entonces observamos un artefacto de borde

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, figsize=(6, 6), tight_layout=True); 

def update(pix_angle=0):
    img = np.zeros(shape=(80, 80));
    for i in range(20):
        for k1 in range(4):
            for k2 in range(4):
                img[i+k1*20, k2*20 + int(i - 2*pix_angle*i/20 + pix_angle)-1] = 1 
    im = ax.matshow(img); 
interact(update, pix_angle=IntSlider_nice(min=0, max=20, value=0));

## Espectro de una imagen sintética

- Un rectangulo en la imagen es un sinc en el espectro

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True); 

def update(width=1):
    img = np.zeros(shape=(80, 80)); #S_img[0, 0] = 0.0;
    img[40-width:40+width, 40-width:40+width] = 1.0; 
    ax[1].set_title("Espectro de amplitud")
    S_img = np.abs(fftpack.fft2(img))
    im = ax[1].imshow(fftpack.fftshift(S_img), extent=[-40, 40, 40, -40])
    ax[0].set_title("Imagen")
    im = ax[0].imshow(img); 

interact(update, width=IntSlider_nice(min=1, max=39, value=0, description="square size"));

## Espectro de una imagen sintética

- Una Gaussiana en el espectro es una Gaussiana en el espacio original

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True); 

def update(bandwidth=1.0):
    x = np.linspace(-100, 100, num=500)
    X, Y = np.meshgrid(x, x)
    img = np.exp(-0.5*(X**2 + Y**2)/bandwidth**2)
    ax[1].set_title("Espectro de amplitud")
    S_img = np.abs(fftpack.fft2(img))
    im = ax[1].imshow(fftpack.fftshift(S_img), extent=[-250, 250, 250, -250])
    ax[0].set_title("Imagen")
    im = ax[0].imshow(img);     
    
interact(update, bandwidth=FloatSlider_nice(min=0.1, max=50.0, value=0, description="Bandwidth"));

## Espectro de una imagen sintética

- Una imagen de ruido blanco y su espectro

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True); 

def update(rseed=1.0):
    np.random.seed(rseed)
    img = np.random.randn(500, 500) 
    ax[1].set_title("Espectro de amplitud")
    S_img = np.log10(1e-6 + np.abs(fftpack.fft2(img)))
    im = ax[1].imshow(fftpack.fftshift(S_img), extent=[-250, 250, 250, -250])
    ax[0].set_title("Imagen")
    im = ax[0].imshow(img); 

interact(update, rseed=IntSlider_nice(min=0, max=100, value=0, description="rseed"));

## Espectro de una imagen sintética

- Una imagen de ruido rojo y su espectro

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True); 

def f(rseed=1.0, gamma=0.0):
    np.random.seed(rseed)
    red_noise = np.random.randn(500, 500) 
    rho = 1-10**-gamma
    #for i in range(2,500):
    #    red_noise[i, :] = rho*red_noise[i-1, :] + red_noise[i, :]
    #    red_noise[:, i] = rho*red_noise[:, i-1] + red_noise[:, i]
    for i in range(2, 500):
        for j in range(2, 500):
            red_noise[i, j] += rho*np.average(red_noise[i-2:i, j-2:j])
    ax[1].set_title("Espectro de amplitud")
    S_img = np.log10(1e-10 + np.abs(fftpack.fft2(red_noise)))
    im = ax[1].imshow(fftpack.fftshift(S_img), extent=[-250, 250, 250, -250])
    ax[0].set_title("Imagen")
    im = ax[0].imshow(red_noise); 

interact(f, rseed=IntSlider_nice(min=0, max=100, value=0, description="rseed"),
        gamma=FloatSlider_nice(min=0, max=3, value=0, description="$\gamma$"));

## Espectro de una imagen natural

- ¿Pueden reconocer a que corresponden las estructuras en el espectro?

In [ ]:
img_building = color2bw(plt.imread('images/building.jpg'))
fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True)
S_img = fftpack.fft2(img_building)
im = ax[0].imshow(fftpack.fftshift(np.log(np.abs(S_img)+1)))
fig.colorbar(im, ax=ax[0], orientation='horizontal')
im = ax[1].imshow(img_building)
fig.colorbar(im, ax=ax[1], orientation='horizontal')

- La fuerte componente horizontal/vertical es en realidad un efecto de borde por la periodicidad de la DFT
- La podemos suavizar usando enventanado

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(np.tile(img_building, (3, 3)))
ax.axis('off');

In [ ]:
win = np.hamming(img_building.shape[0]).reshape(-1, 1)
win = np.dot(win, win.T)

fig, ax = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True)
S_img = fftpack.fft2(win*img_building)
im = ax[0].imshow(fftpack.fftshift(np.log(np.abs(S_img)+1)))
fig.colorbar(im, ax=ax[0], orientation='horizontal')
im = ax[1].imshow(img_building)
fig.colorbar(im, ax=ax[1], orientation='horizontal')

## Filtrado de imágenes en el dominio de frecuencia

- Una multiplicación en el espacio de frecuencia equivale a una convolución en el espacio original
- ¿Qué ocurre con la imagen original cuando énmascaramos una parte del espectro?

In [ ]:
img_doge = color2bw(plt.imread('images/doge.jpg')) 

plt.figure(figsize=(9, 7))
plt.imshow(img_doge)
plt.colorbar(orientation='horizontal');

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5), tight_layout=True)
S_img = fftpack.fft2(img_doge)
im = ax[0].imshow(fftpack.fftshift(np.log(1.+np.abs(S_img))))
fig.colorbar(im, ax=ax[0], orientation='horizontal')
im = ax[1].imshow(fftpack.fftshift(np.angle(S_img)))  # arctan(imag/real)
fig.colorbar(im, ax=ax[1], orientation='horizontal');

## Espectro de magnitud y de fase

- La **magnitud espectral** guarda información de la amplitud de los componentes
- La **fase espectral** guarda información del desfase (posición) de los componentes
- ¿Podemos reconstruir usando sólo uno de los espectros?

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 8), tight_layout=True)
for ax_ in ax.ravel():
    ax_.axis('off')

ax[0, 0].imshow(img_doge);
S_dog = fftpack.fft2(img_doge)
reconstruct = lambda S: np.real(fftpack.ifft2(S))
ax[1, 0].imshow(fftpack.fftshift(reconstruct(np.abs(S_dog))));
ax[1, 1].imshow(reconstruct(np.exp(1j*np.angle(S_dog, deg=False))));

¿Y si intercambiamos la fase y magnitud de dos imágenes de igual tamaño?

In [ ]:
img_inst = color2bw(plt.imread("images/InsInformatica.jpg"))  
fig, ax = plt.subplots(2, 2, figsize=(9, 7), tight_layout=True)
for ax_ in ax.ravel():
    ax_.axis('off')

ax[0, 0].imshow(img_doge);
ax[0, 1].imshow(img_inst); 
S_inf = fftpack.fft2(img_inst)
rec_doge = fftpack.ifft2(np.abs(S_dog)*np.exp(1j*np.angle(S_inf, deg=False)))
rec_inst = fftpack.ifft2(np.abs(S_inf)*np.exp(1j*np.angle(S_dog, deg=False)))
ax[1, 0].set_title('Amplitud doge\nAngulo instituto')
ax[1, 0].imshow(np.abs(rec_doge)); 
ax[1, 1].set_title('Amplitud instituto\nAngulo doge')
ax[1, 1].imshow(np.abs(rec_inst)); 

## Filtro pasa-bajos: Eliminando las frecuencias altas

1) Multiplicando el espectro por una ventana rectangular

In [ ]:
cy, cx = img_doge.shape[0]/2, img_doge.shape[1]/2
x = np.arange(0, img_doge.shape[1]); y = np.arange(0, img_doge.shape[0]);
X, Y = np.meshgrid(x, y)

def f(sigma):
    fig = plt.figure(figsize=(18, 10)); ax = fig.add_subplot(1, 2 ,1);
    mask = np.zeros(shape=X.shape)
    mask[int(cy-sigma):int(cy+sigma), int(cx-sigma):int(cx+sigma)] = 1
    im = ax.imshow(fftpack.fftshift(np.log(1+np.abs(S_img)))*mask, cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    ax = fig.add_subplot(1, 2 ,2)
    im = ax.imshow(np.real(fftpack.ifft2(fftpack.ifftshift(fftpack.fftshift(S_img)*mask))), cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    fig.tight_layout(); flush_figures();
interact(f, sigma=IntSlider(min=1, max=200.0, value=200, description="Width", layout=slider_layout));

2) Multiplicando el espectro por una ventana Gaussiana

In [ ]:
def f(sigma):
    fig = plt.figure(figsize=(12, 8)); ax = fig.add_subplot(1, 2 ,1);
    mask = 1e-8 + np.exp(-(((X-cx)/sigma)**2 + ((Y-cy)/sigma)**2))
    im = ax.imshow(fftpack.fftshift(np.log(1+np.abs(S_img)))*mask, cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    ax = fig.add_subplot(1, 2 ,2)
    im = ax.imshow(np.real(fftpack.ifft2(fftpack.ifftshift(fftpack.fftshift(S_img)*mask))), cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    fig.tight_layout(); flush_figures();
interact(f, sigma=FloatSlider(min=1, max=200.0, value=200, description="$\sigma$", layout=slider_layout));

## Filtro pasa-altos: Eliminando las frecuencias bajas

In [ ]:
def f(sigma=1):
    fig = plt.figure(figsize=(12, 8)); ax = fig.add_subplot(1, 2 ,1);
    mask = 1.0  - np.exp(-(((X-cx)/sigma)**2 + ((Y-cy)/sigma)**2)) 
    im = ax.imshow(fftpack.fftshift(np.log(1.0+np.abs(S_img)))*mask, cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    ax = fig.add_subplot(1, 2 ,2)
    im = ax.imshow(np.real(fftpack.ifft2(fftpack.ifftshift(fftpack.fftshift(S_img)*mask))), cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    fig.tight_layout(); flush_figures();
interact(f, sigma=FloatSlider(min=1, max=100.0, value=1, description="$\sigma$", layout=slider_layout));

## Filtro pasa-banda y rechaza-banda

In [ ]:
def f(sigma1=1, sigma2=1):
    fig = plt.figure(figsize=(12, 8)); ax = fig.add_subplot(1, 2 ,1);
    mask1 = np.exp(-(((X-cx)/sigma1)**2 + ((Y-cy)/sigma1)**2)) 
    mask2 = np.exp(-(((X-cx)/sigma2)**2 + ((Y-cy)/sigma2)**2)) 
    im = ax.imshow(fftpack.fftshift(np.log(1.0+np.abs(S_img)))*(mask1-mask2), cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    ax = fig.add_subplot(1, 2 ,2)
    im = ax.imshow(np.real(fftpack.ifft2(fftpack.ifftshift(fftpack.fftshift(S_img)*(mask1-mask2)))), 
                   cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    fig.tight_layout(); flush_figures();
interact(f, sigma1=FloatSlider(min=1, max=200.0, value=200, description="$\sigma_1$", layout=slider_layout),
        sigma2=FloatSlider(min=1, max=200.0, value=1, description="$\sigma_2$", layout=slider_layout));

## Filtros y convolucion 2D

- ¿Cómo se ve una convolución en dos dimensiones?
- El filtro de convolución es también llamado kernel

<img src="images/filter2D_convolution.gif" width="600">


- ¿Qué hacen estos kernels/filtros?

$$
\begin{pmatrix}
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 \\
\end{pmatrix} \frac{1}{25} 
\qquad
\begin{pmatrix}
0.018 & 0.082 & 0.1353 & 0.082 & 0.018 \\
0.082 & 0.3678 & 0.6065 & 0.3678 & 0.082 \\
0.1353 & 0.6065 & 1 & 0.6065 & 0.1353 \\
0.082 & 0.3678 & 0.6065 & 0.3678 & 0.082 \\
0.018 & 0.082 & 0.1353 & 0.082 & 0.018 \\
\end{pmatrix} \frac{1}{\sqrt{2\pi}}
$$




In [ ]:
from scipy.signal import fftconvolve, convolve2d
def f(size=1):
    fig = plt.figure(figsize=(12, 8)); 
    ax = fig.add_subplot(1, 2 ,1);
    kernel = np.ones(shape=(size, size))/size**2
    img_filtered1 = fftconvolve(img_doge, kernel, mode='same');
    img_filtered2 = convolve2d(img_doge, kernel, mode='same');
    im = ax.imshow(img_filtered1, cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    ax = fig.add_subplot(1, 2 ,2)
    im = ax.imshow(img_filtered2, cmap=plt.cm.Greys_r)
    fig.colorbar(im, ax=ax, orientation='horizontal')
    fig.tight_layout(); flush_figures();
interact(f, size=IntSlider(min=1, max=100.0, value=1, description="Size", layout=slider_layout));

In [ ]:
def f(size=1):
    kernel = np.ones(shape=(size, size))/size**2
    %time img_filtered1 = fftconvolve(img_doge, kernel, mode='full');
    %time img_filtered2 = convolve2d(img_doge, kernel, mode='same');    
interact(f, size=IntSlider(min=1, max=100.0, value=1, description="Size", layout=slider_layout));

## Filtros y convolucion 2D

- ¿Qué hacen estos filtros/kernels?
- Se conocen como filtro sobel horizontal y vertical

<img src="images/filtro_gradient.gif" width="600">

In [ ]:
fig = plt.figure(figsize=(12, 12))
sobelx = fftconvolve(img_doge, [[-1, 0, 1],[-2, 0, 2], [-1, 0, 1]], mode='full')
ax =  plt.subplot2grid((2, 2), (0, 0))
ax.matshow(sobelx, cmap=plt.cm.Greys_r); ax.axis('off')
sobely = fftconvolve(img_doge, [[-1, -2, -1],[0, 0, 0], [1, 2, 1]], mode='full')
ax = plt.subplot2grid((2, 2), (0, 1))
ax.matshow(sobely, cmap=plt.cm.Greys_r);ax.axis('off')
ax = plt.subplot2grid((2, 2), (1, 0), colspan=2)
ax.matshow(np.sqrt(sobely**2 + sobelx**2)[3:-3,3:-3], cmap=plt.cm.Greys_r); ax.axis('off');
plt.tight_layout();

## Ruido y restauración de imágenes

Existen distintos tipos de ruido que pueden afectar una imagen

- Ruido térmico, ruido de lectura, ruido eléctronico: Se modela tipicamente como ruido blanco Gaussiano
- Ruido de disparo (shot noise): Ruido fotónico cuando hay pocas cuentas. Se modela como ruido Poissoniano
- Ruido periódico: Ruido causado por interferencias
- Ruido sal y pimienta: Ruido impulsivo que puede ocurrir por problemas de transmisión o conversión AD

El filtrado en el espacio de frecuencias puede usarse para disminuir el ruido

### Ejemplo: ruido blanco Gaussiano

In [ ]:
def f(strengh):
    fig = plt.figure(figsize=(10, 8)); ax = fig.add_subplot(1, 1 ,1);
    doge_corrupted = img_doge + strengh*np.random.randn(img_doge.shape[0], img_doge.shape[1])
    ax.matshow(doge_corrupted, cmap=plt.cm.Greys_r);
    fig.tight_layout(); flush_figures();
interact(f, strengh=FloatSlider(min=1, max=100.0, value=0, description="Strengh", layout=slider_layout));

### Ejemplo: ruido Poissoniano

In [ ]:
def f(seed):
    fig = plt.figure(figsize=(10, 8)); ax = fig.add_subplot(1, 1 ,1);
    np.random.seed(seed)
    doge_corrupted = np.random.poisson(img_doge, size=img_doge.shape)
    #doge_corrupted = 255*doge_corrupted/np.amax(doge_corrupted)
    ax.matshow(doge_corrupted, cmap=plt.cm.Greys_r);
    fig.tight_layout(); flush_figures();
interact(f, seed=IntSlider(min=1, max=200.0, value=0, description="seed", layout=slider_layout));

### Ejemplo: ruido periódico

In [ ]:
def f(strengh, frequency):
    fig = plt.figure(figsize=(10, 8)); ax = fig.add_subplot(1, 1 ,1);
    doge_corrupted = img_doge + strengh*np.cos(2.0*np.pi*frequency*Y/480)
    ax.matshow(doge_corrupted, cmap=plt.cm.Greys_r);
    fig.tight_layout(); flush_figures();

interact(f, strengh=FloatSlider(min=1, max=100.0, value=50, description="Strengh", layout=slider_layout),
        frequency=FloatSlider(min=0.0, max=80, step=0.01, 
                              value=0, description="Frequency", layout=slider_layout));

### Ruido impulsivo

In [ ]:
def f(strengh):
    fig = plt.figure(figsize=(10, 8)); ax = fig.add_subplot(1, 1 ,1);    
    noise = np.random.randint(low=0, high=strengh, size=img_doge.shape)
    doge_corrupted  = np.where(noise == 0, 0, img_doge)
    doge_corrupted = np.where(noise == strengh-1, 255, doge_corrupted)
    ax.matshow(doge_corrupted, cmap=plt.cm.Greys_r);
    fig.tight_layout(); flush_figures();

interact(f, strengh=IntSlider(min=1, max=255, value=255, description="Strengh", layout=slider_layout));

### Ejemplo: Eliminando ruido periódico

In [ ]:
def f(frequency, size):
    fig = plt.figure(figsize=(12, 10), dpi=150); ax = fig.add_subplot(2, 2 ,1); 
    x = np.arange(0, img_doge.shape[1]); y = np.arange(0, img_doge.shape[0]);
    X, Y = np.meshgrid(x, y)
    periodic_noise = np.cos(2.0*np.pi*60*Y/480)
    doge_corrupted = img_doge + 100*periodic_noise
    ax.matshow(doge_corrupted, cmap=plt.cm.Greys_r);
    ax = fig.add_subplot(2, 2 ,2);  
    S_img = fftpack.fftshift(fftpack.fft2(doge_corrupted))
    S_img[240-size-frequency:240+size-frequency, 320-size:320+size] = 0 
    S_img[240-size+frequency:240+size+frequency, 320-size:320+size] = 0 
    ax.matshow(np.log(1+np.abs(S_img))[120:-120, 160:-160])
    ax = fig.add_subplot(2, 2, 3);
    ax.matshow(np.mean(img_doge) + np.abs(fftpack.ifft2(fftpack.ifftshift(S_img))), cmap=plt.cm.Greys_r)
    fig.tight_layout(); flush_figures();

interact(f, size=IntSlider(min=1, max=20, value=10, description="Mask size", layout=slider_layout),
        frequency=IntSlider(min=0.0, max=100, value=100, description="Mask position", layout=slider_layout));

## Restauración de imágenes y deconvolución

- Una imagen observada $g(x,y)$ se puede modelar como
$$
g(x,y) =  f(x, y) * h(x, y) + n(x,y)
$$
donde $f(x,y)$ es la imagen original, $h(x,y)$ es la Point Spread Function (PSF) y $n(x,y)$ es ruido aditivo.

- La PSF modela la respuesta al impulso del sistema que captura la imagen. Usualmente tiene una forma Gaussiana, es decir suaviza la imagen original.

### Ejemplo: PSF de un telescopio
<table><tr><td>
    <img src="images/psf3.png" width="250"></td><td><img src="images/PSF1.png" width="550">
</td></tr></table>
<center><img src="images/PSF2.jpeg" width="600"></center>


## Deconvolución

- Deconvolución se refiere al proceso de recuperar $f(x,y)$ a partir de $g(x,y)$ usando supuestos sobre $h(x,y)$ y $n(x,y)$

- Si trabajamos en frecuencia:

$$
G(f_1, f_2) = F(f_1, f_2) \cdot H(f_1, f_2) + N(f_1, f_2)
$$

- Si ignoramos N podemos recuperar F usando un filtro inverso
$$
F(f_1, f_2) = G(f_1, f_2) / H(f_1, f_2)
$$
- Problema resuelto?

Sea una imagen como la siguiente

In [ ]:
img_seadoge = np.dot(plt.imread('images/lobo.jpg'), [0.299, 0.587, 0.114])  
plt.figure(figsize=(10, 7))
plt.imshow(img_seadoge, cmap=plt.cm.Greys_r);

Asumamos que nuestro sistema de captura tiene una PSF Gaussiana con $\sigma=2$ y ruido blanco Gaussiano con desviación estándar 20

In [ ]:
x = np.linspace(-5, 5, num=11)
X, Y = np.meshgrid(x, x)
sigma = 2
psf = np.exp(-0.5*(X)**2/sigma**2 - 0.5*Y**2/sigma**2)/(2.0*np.pi*sigma**2)
img_seadoge_observed = fftconvolve(img_seadoge, psf, mode='same') \
+ 20*np.random.randn(img_seadoge.shape[0], img_seadoge.shape[1])
img_min, img_max = np.amin(img_seadoge_observed), np.amax(img_seadoge_observed)
img_seadoge_observed = 255*(img_seadoge_observed - img_min)/(img_max - img_min)
plt.figure(figsize=(10, 7))
plt.imshow(img_seadoge_observed, cmap=plt.cm.Greys_r);

- El filtro inverso es débil contra el ruido
- Hay que tener cuidado al invertir valores cercanos a cero

In [ ]:
def f(sigma, tol):
    fig = plt.figure(figsize=(14, 8))
    ax =  fig.add_subplot(2, 2, 1)
    ax.matshow(img_seadoge_observed, cmap=plt.cm.Greys_r); ax.axis('off')
    S_img = fftpack.fft2(img_seadoge_observed/255)
    ax =  fig.add_subplot(2, 2, 2)
    ax.matshow(fftpack.fftshift(np.log(1.0+np.abs(S_img))), cmap=plt.cm.Greys_r); ax.axis('off')
    ax =  fig.add_subplot(2, 2, 3)
    fx, fy = fftpack.fftfreq(n=1000, d=1), fftpack.fftfreq(n=600, d=1)
    Fx, Fy = np.meshgrid(fx, fy)
    inv_psf =  1.0/np.exp(-2*np.pi**2*(Fx**2 + Fy**2)*sigma**2)
    inv_psf[inv_psf > 1.0/10**tol] = 0.0
    ax.matshow(fftpack.fftshift(inv_psf), cmap=plt.cm.Greys_r); ax.axis('off')
    ax =  fig.add_subplot(2, 2, 4)
    WF = S_img*inv_psf; 
    ax.matshow(np.abs(fftpack.ifft2(WF)), cmap=plt.cm.Greys_r); ax.axis('off'); 
    plt.tight_layout(pad=0.1);flush_figures(); 
    
interact(f, sigma=FloatSlider(min=0, max=10.0, value=2, description="$\sigma$", layout=slider_layout),
        tol=FloatSlider(min=-5, max=0, value=-6, description="log(tolerance)", layout=slider_layout),);

## Filtro de Wiener

Es el filtro óptimo que resulta de minimizar el error cuadrático medio entre la imagen observada y la imagen original
$$
W = \frac{H^{*}}{|H|^2 + S_n/S_f},
$$
donde $S_n$ es la densidad espectral del ruido y $S_f$ es la densidad espectral de la señal original.

En general no conocemos el factor $S_n/S_f = 1/SNR$ pero podemos hacer supuestos

In [ ]:
def f(sigma, K):
    fig = plt.figure(figsize=(16, 7))
    ax = fig.add_subplot(1, 2, 1)
    ax.matshow(img_seadoge_observed, cmap=plt.cm.Greys_r)
    ax = fig.add_subplot(1, 2, 2)
    fx, fy = fftpack.fftfreq(n=1000, d=1), fftpack.fftfreq(n=600, d=1)
    Fx, Fy = np.meshgrid(fx, fy)
    inv_psf =  np.exp(-2*np.pi**2*(Fx**2 + Fy**2)*2**2)
    #inv_psf[inv_psf > 1.0/tol] = 0.0
    S_img = fftpack.fft2(img_seadoge_observed/255)
    WF = S_img*np.conj(inv_psf)/(np.abs(inv_psf)**2 + 10**K)
    ax.matshow(np.abs(fftpack.ifft2(WF)), cmap=plt.cm.Greys_r);
    plt.tight_layout(); flush_figures(); 
    
interact(f, sigma=FloatSlider(min=0, max=10, value=2, description="$\sigma$", layout=slider_layout),
         K=FloatSlider(min=-5, max=5, value=0, description="K=Sn/Sf", layout=slider_layout));

In [ ]:
from skimage.restoration import wiener, unsupervised_wiener
def f(balance):
    img_seadoge_recovered = wiener(img_seadoge_observed/255.0, psf=psf, balance=10**balance)
    fig = plt.figure(figsize=(16, 7))
    ax = fig.add_subplot(1, 2, 1)
    ax.matshow(img_seadoge_observed, cmap=plt.cm.Greys_r)
    ax = fig.add_subplot(1, 2, 2)
    ax.matshow(img_seadoge_recovered, cmap=plt.cm.Greys_r);
    plt.tight_layout(); flush_figures(); 
    
interact(f, balance=FloatSlider(min=-5, max=5, value=0, description="Regularizacion", layout=slider_layout));

In [ ]:
img_seadoge_recovered = unsupervised_wiener(img_seadoge_observed/255.0, psf=psf)[0]
fig = plt.figure(figsize=(16, 7))
ax = fig.add_subplot(1, 2, 1)
ax.matshow(img_seadoge_observed, cmap=plt.cm.Greys_r)
ax = fig.add_subplot(1, 2, 2)
ax.matshow(img_seadoge_recovered, cmap=plt.cm.Greys_r);
plt.tight_layout();